In [1]:
import pandas as pd
import numpy as np
import re
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.compose import ColumnTransformer

# Rent

## 加载与查看数据

In [2]:
original_data = pd.read_csv('ruc_Class25Q2_train_rent.csv')
original_test = pd.read_csv('ruc_Class25Q2_test_rent.csv')
def clean_column_name(col_name):
    return col_name.replace(' ', '')
original_data = original_data.rename(columns=clean_column_name)
original_test = original_test.rename(columns=clean_column_name)
print("=== 数据基本信息 ===")
original_data.info()

/tmp/ipykernel_2156/2367139569.py:1: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  original_data = pd.read_csv('ruc_Class25Q2_train_rent.csv')


=== 数据基本信息 ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98899 entries, 0 to 98898
Data columns (total 46 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   城市       98899 non-null  int64  
 1   户型       98898 non-null  object 
 2   装修       25410 non-null  object 
 3   Price    98899 non-null  float64
 4   楼层       98894 non-null  object 
 5   面积       98899 non-null  object 
 6   朝向       98894 non-null  object 
 7   交易时间     98899 non-null  object 
 8   付款方式     80476 non-null  object 
 9   租赁方式     98899 non-null  object 
 10  电梯       98895 non-null  object 
 11  车位       24764 non-null  object 
 12  用水       81159 non-null  object 
 13  用电       81575 non-null  object 
 14  燃气       94317 non-null  object 
 15  采暖       34412 non-null  object 
 16  租期       51966 non-null  object 
 17  配套设施     68448 non-null  object 
 18  lon      98899 non-null  float64
 19  lat      98899 non-null  float64
 20  年份       98899 non-null  float64
 2

In [3]:
print("=== 数值型变量描述性统计 ===")
original_data.describe()

=== 数值型变量描述性统计 ===


,城市,Price,lon,lat,年份,区县,板块,容积率,停车位,coord_x,coord_y
count,98899.000000,9.889900e+04,98899.000000,98899.000000,98899.000000,94222.000000,93755.000000,74819.000000,73420.000000,94268.000000,94268.000000
mean,4.322157,5.829090e+05,115.752394,31.420651,2021.643010,70.243054,588.404096,3.060876,1285.232866,115.861282,31.338518
std,3.321254,6.218424e+05,5.578828,6.378794,0.479114,35.832629,344.070325,1.751343,1576.732337,5.595015,6.365706
min,0.000000,1.793807e+04,103.482606,23.025849,2021.000000,3.000000,1.000000,0.020000,1.000000,103.503817,23.025849
25%,2.000000,2.416282e+05,114.357093,24.201337,2021.000000,38.000000,298.000000,2.000000,300.000000,114.360217,24.183552
50%,3.000000,3.949369e+05,117.156819,32.163991,2022.000000,70.000000,585.000000,2.670000,779.000000,117.310723,32.162232
75%,7.000000,7.248954e+05,121.644325,37.606197,2022.000000,95.000000,895.000000,3.800000,1590.000000,121.666901,32.705437
max,11.000000,1.540419e+07,122.966660,42.189642,2022.000000,131.000000,1186.000000,30.000000,8700.000000,122.966660,42.189642


In [4]:
print(original_data.head(10))

   城市      户型   装修         Price       楼层       面积  朝向        交易时间 付款方式 租赁方式  \
0   0  1室1厅1卫  精装修  6.546465e+05     4/6层   36.42㎡   西  2024-11-28  季付价   整租   
1   0  1室1厅1卫  精装修  6.654121e+05     4/6层   41.00㎡   南  2024-10-30  季付价   整租   
2   0  1室1厅1卫  精装修  7.782228e+05    1/18层   37.36㎡   北  2024-11-12  季付价   整租   
3   0  3室1厅2卫  精装修  6.120850e+05    1/10层   55.42㎡   南  2024-10-14  季付价   整租   
4   0  1室1厅1卫  精装修  9.947321e+05   18/18层   49.30㎡   南  2024-12-08  季付价   整租   
5   0  1室1厅1卫  精装修  9.266278e+05   17/17层   42.00㎡   北  2024-10-13  季付价   整租   
6   0  1室1厅1卫  精装修  8.905057e+05   17/17层   42.00㎡   北  2024-10-13  季付价   整租   
7   0  1室1厅1卫  精装修  8.725905e+05    5/17层   42.12㎡   北  2024-10-22  季付价   整租   
8   0  1室1厅1卫  精装修  9.496648e+05    5/17层   42.12㎡   北  2024-10-22  季付价   整租   
9   0    3室2厅  NaN  2.052022e+06  中楼层/25层  176.00㎡  西南  2024-06-01  季付价   整租   

   ...     供水        供暖     供电            燃气费       供热费    停车位  \
0  ...     民水      集中供暖     民电       2.61元/m³  24-30元

## 数据预处理

### 删除部分行/列

In [5]:
# 删除缺失值过多、相关性低、难以结构化、重复的列
missing_counts = original_data.isnull().sum()
missing_sorted = missing_counts.sort_values(ascending=False)
print("=== 列名及缺失值数量（从高到低） ===")
for col, count in missing_sorted.items():
    print(f"{col}: {count}")

=== 列名及缺失值数量（从高到低） ===
车位: 74135
装修: 73489
供热费: 70064
环线位置: 69663
采暖: 64487
供暖: 61204
物业办公电话: 60692
租期: 46933
配套设施: 30451
物业公司: 28753
开发商: 26984
建筑年代: 26149
停车费用: 26003
停车位: 25479
燃气费: 25057
绿化率: 24402
容积率: 24080
物业费: 22159
供电: 21200
供水: 21194
建筑结构: 20741
物业类别: 20124
产权描述: 20124
付款方式: 18423
用水: 17740
用电: 17324
板块: 5144
区县: 4677
楼栋总数: 4631
房屋总数: 4631
coord_y: 4631
coord_x: 4631
燃气: 4582
楼层: 5
朝向: 5
电梯: 4
户型: 1
城市: 0
交易时间: 0
租赁方式: 0
Price: 0
面积: 0
lat: 0
lon: 0
年份: 0
客户反馈: 0


In [6]:
columns_to_drop = ['车位', '物业类别', '开发商', '物业公司', '物业办公电话', '产权描述', '供水', '供暖', '供电', '燃气费', '供热费', '停车费用', 'coord_x', 'coord_y', '客户反馈']
data_1 = original_data.drop(columns_to_drop, axis=1)
test_1 = original_test.drop(columns_to_drop, axis=1)

In [7]:
# 删除缺失值过多的样本
sample_missing_counts = data_1.isnull().sum(axis=1)
print("=== 样本统计 ===")
print(f"总样本数量: {len(data_1)}")
print(f"总列数: {data_1.shape[1]}")
missing_value_distribution = sample_missing_counts.value_counts().sort_index()

print("\n=== 缺失值统计 ===")
print("缺失值数量 | 样本数量")
print("-" * 20)
for missing_count, sample_count in missing_value_distribution.items():
    print(f"{missing_count:^11} | {sample_count:^8}")

=== 样本统计 ===
总样本数量: 98899
总列数: 31

=== 缺失值统计 ===
缺失值数量 | 样本数量
--------------------
     0      |   3095  
     1      |  10657  
     2      |  13946  
     3      |  16712  
     4      |  12255  
     5      |   9789  
     6      |   4735  
     7      |   3603  
     8      |   3672  
     9      |   5832  
    10      |   3158  
    11      |   3522  
    12      |   2190  
    13      |   1853  
    14      |   1064  
    15      |   889   
    16      |   471   
    17      |   384   
    18      |   299   
    19      |   773   


In [8]:
threshold = 18
data_2 = data_1.copy()
test_2 = test_1.copy()
data_2 = data_2[sample_missing_counts < threshold]
data_2 = data_2.reset_index(drop=True)
print("=== 处理后样本量统计 ===")
print(f"原样本数量: {len(data_1)}")
print(f"删除样本数量: {len(data_1) - len(data_2)}  删除比例：{(len(data_1) - len(data_2))/len(data_1)*100:.2f}%")
print(f"处理后样本数量: {len(data_2)}")

=== 处理后样本量统计 ===
原样本数量: 98899
删除样本数量: 1072  删除比例：1.08%
处理后样本数量: 97827


### 数据提取与编码

In [9]:
data_3 = pd.DataFrame()
test_3 = pd.DataFrame()

In [10]:
# 城市、区县、板块
data_3[['城市', '区县', '板块']] = data_2[['城市', '区县', '板块']].copy()
test_3[['城市', '区县', '板块']] = test_2[['城市', '区县', '板块']].copy()

In [11]:
# 环线位置
data_3['环线位置'] = np.nan
test_3['环线位置'] = np.nan

# 定义汉字数字到阿拉伯数字的映射字典
hanzi_to_num = {
    '一': 1, '二': 2, '三': 3, '四': 4, '五': 5,
    '六': 6, '七': 7, '八': 8, '九': 9, '十': 10
}

def process_ring_value(s):
    if pd.isna(s):
        return np.nan
    
    # 转换为字符串并去除首尾空格（避免格式问题）
    s = str(s).strip()
    
    # 内环内→1，外环外→6
    if s == '内环内':
        return 1
    if s == '外环外':
        return 6

    # 内环至中环→2，内环至外环→3，中环至外环→4
    if s == '内环至中环':
        return 2
    if s == '内环至外环':
        return 3 
    if s == '中环至外环':
        return 4
    
    # x环内→x-1，x环外→x
    if '环内' in s:
        x_hanzi = s[0]
        if x_hanzi in hanzi_to_num:
            return hanzi_to_num[x_hanzi] - 1
    elif '环外' in s:
        x_hanzi = s[0]
        if x_hanzi in hanzi_to_num:
            return hanzi_to_num[x_hanzi]
    
    # x至y环→(x+y)/2向下取整
    first_char = s[0] if len(s) > 0 else ''
    third_char = s[2] if len(s) > 2 else ''
    if first_char in hanzi_to_num and third_char in hanzi_to_num:
        return int((hanzi_to_num[first_char] + hanzi_to_num[third_char])/2)
    
    # 其他未覆盖的情况返回缺失值
    return np.nan

data_3['环线位置'] = data_2['环线位置'].apply(process_ring_value)
test_3['环线位置'] = test_2['环线位置'].apply(process_ring_value)

In [12]:
# Price
data_3['Price'] = data_2['Price'].copy()

In [13]:
# 户型
data_2['户型'] = data_2['户型'].str.replace('房间', '室')
data_2['户型'] = data_2['户型'].str.replace('居室', '室')
data_3[['室', '厅', '卫']] = [np.nan, np.nan, np.nan]
test_2['户型'] = test_2['户型'].str.replace('房间', '室')
test_2['户型'] = test_2['户型'].str.replace('居室', '室')
test_3[['室', '厅', '卫']] = [np.nan, np.nan, np.nan]

keywords = ['室', '厅', '卫']

def process_house_layout(s, keyword):
    if pd.isna(s):
        return np.nan

    # 正则匹配
    pattern = r'(\d+)' + keyword
    match = re.search(pattern, str(s))
    if match:
        return int(match.group(1))
    else:
        return np.nan

for keyword in keywords:
    data_3[keyword] = data_2['户型'].apply(lambda x: process_house_layout(x, keyword))
    test_3[keyword] = test_2['户型'].apply(lambda x: process_house_layout(x, keyword))

In [14]:
# 楼层
floor_to_num = {
    '地下室': -1.0, '低楼层': 1/3, '中楼层': 2/3, '高楼层': 1.0
}

def process_floor(s):
    if pd.isna(s):
         return pd.Series([np.nan, np.nan], index=['所在楼层', '总层数'])
        
    floor = np.nan
    floor_tol = np.nan

    pattern = r'地下\s*(\d+)\s*层'
    match = re.search(pattern, str(s))
    if match:
        floor = -int(match.group(1))
        return pd.Series([floor, floor_tol], index=['所在楼层', '总层数'])
        
    s_1 = str(s).split('/')[0]
    s_2 = str(s).split('/')[1]
    pattern = r'(\d+)\s*层'
    match = re.search(pattern, str(s_2))
    if match:
        floor_tol = int(match.group(1))

    if s_1 == '地下室':
        floor = -1
    elif s_1.strip() in floor_to_num:
        floor = int(floor_to_num[s_1.strip()] * floor_tol)
    else:
        floor = int(s_1.strip())
    return pd.Series([floor, floor_tol], index=['所在楼层', '总层数'])
    
data_3 = pd.concat([data_3, data_2['楼层'].apply(process_floor)], axis=1)
test_3 = pd.concat([test_3, test_2['楼层'].apply(process_floor)], axis=1)

In [15]:
# 面积
data_3['面积'] = np.nan
test_3['面积'] = np.nan

def process_area(s):
    if pd.isna(s):
        return np.nan
    pattern = r'(\d+\.?\d*)\s*㎡'
    match = re.search(pattern, str(s))
    if match:
        return float(match.group(1))
    else:
        return np.nan

data_3['面积'] = data_2['面积'].apply(process_area)
test_3['面积'] = test_2['面积'].apply(process_area)

In [16]:
# 朝向
directions = ['东', '南', '西', '北', '东南', '东北', '西南', '西北']

def process_directions(s):
    if pd.isna(s):
        return pd.Series(np.nan, index=[f'朝向_{d}' for d in directions])
    
    parts = [p.strip() for p in str(s).split() if p.strip()]
    result = pd.Series(0, index=[f'朝向_{d}' for d in directions])
    
    for direction in directions:
        col_name = f'朝向_{direction}'
        result[col_name] = parts.count(direction)
    return result

data_3 = pd.concat([data_3, data_2['朝向'].apply(process_directions)], axis=1)
test_3 = pd.concat([test_3, test_2['朝向'].apply(process_directions)], axis=1)

In [17]:
# 装修、付款方式、租赁方式、电梯、用水、用电、燃气、采暖
data_3['装修'] = data_2['装修'].fillna('非精装修')
data_3['付款方式'] = data_2['付款方式'].apply(lambda x: x if '付价' in str(x) else '')
data_3[['租赁方式', '电梯', '用水', '用电', '燃气', '采暖']] = data_2[['租赁方式', '电梯', '用水', '用电', '燃气', '采暖']].copy()

test_3['装修'] = test_2['装修'].fillna('非精装修')
test_3['付款方式'] = test_2['付款方式'].apply(lambda x: x if '付价' in str(x) else '')
test_3[['租赁方式', '电梯', '用水', '用电', '燃气', '采暖']] = test_2[['租赁方式', '电梯', '用水', '用电', '燃气', '采暖']].copy()

In [18]:
# 交易时间
data_3['交易时间'] = data_2['交易时间'].str.split('-').str[0].astype('Int64')
test_3['交易时间'] = test_2['交易时间'].str.split('-').str[0].astype('Int64')

In [19]:
# 配套设施
facilities_list = (data_2['配套设施'].dropna().str.split('、').explode().str.strip().unique().tolist())
facilities_list.remove('')
facilities_list.remove('天然气')

for facility in facilities_list:
    data_3[facility] = np.nan
    test_3[facility] = np.nan

def process_facility(s, item):
    if pd.isna(s):
        return np.nan

    if item in s:
        return 1
    else:
        return 0
for facility in facilities_list:
    data_3[facility] = data_2['配套设施'].apply(lambda x: process_facility(x, facility))
    test_3[facility] = test_2['配套设施'].apply(lambda x: process_facility(x, facility))

In [20]:
# lon、lat、年份
data_3[['lon', 'lat', '年份']] = data_2[['lon', 'lat', '年份']].copy()
test_3[['lon', 'lat', '年份']] = test_2[['lon', 'lat', '年份']].copy()

In [21]:
# 建筑年代
def process_construct_year(s):
    if pd.isna(s):
        return pd.Series(np.nan, index=['建筑年代_1', '建筑年代_2'])

    s_year = s.split('年')[0]
    s_len = len(s_year.split('-'))
    left = 0
    right = 0
    if s_len > 1:
        left = int(s_year.split('-')[0])
        right = int(s_year.split('-')[1])
    else:
        left = int(s_year)
        right = int(s_year)
    return pd.Series([left, right], index=['建筑年代_1', '建筑年代_2'])
    
data_3 = pd.concat([data_3, data_2['建筑年代'].apply(process_construct_year)], axis=1)
test_3 = pd.concat([test_3, test_2['建筑年代'].apply(process_construct_year)], axis=1)

In [22]:
# 房屋总数、楼栋总数、绿化率、容积率
data_3['房屋总数'] = data_2['房屋总数'].str.replace('户', '').astype('Int64')
data_3['楼栋总数'] = data_2['楼栋总数'].str.replace('栋', '').astype('Int64')
data_3['绿化率'] = data_2['绿化率'].str.replace('%', '').astype('Float64')/100
data_3['容积率'] = data_2['容积率']/100

test_3['房屋总数'] = test_2['房屋总数'].str.replace('户', '').astype('Int64')
test_3['楼栋总数'] = test_2['楼栋总数'].str.replace('栋', '').astype('Int64')
test_3['绿化率'] = test_2['绿化率'].str.replace('%', '').astype('Float64')/100
test_3['容积率'] = test_2['容积率']/100

In [23]:
# 建筑结构（塔楼/平房/板楼/塔板结合）
structures = ['塔楼', '平房', '板楼', '塔板结合']

def process_structure(s, item):
    if pd.isna(s):
        return np.nan

    if item in s:
        return 1
    else:
        return 0

for structure in structures:
    data_3[structure] = data_2['建筑结构'].apply(lambda x: process_structure(x, structure))
    test_3[structure] = test_2['建筑结构'].apply(lambda x: process_structure(x, structure))

In [24]:
# 租期
timeunit_to_num = {
    '月': 1,  '年': 12
}

def process_time(s):
    if pd.isna(s):
        return np.nan

    unit = 1
    for timeunit in timeunit_to_num.keys():
        if timeunit in s:
            unit = timeunit_to_num[timeunit]
            break

    num = np.nan
    numbers = re.findall(r'\d+', s)
    if numbers:
        numbers_list = [int(x) for x in numbers]
        num = sum(numbers_list) * unit / len(numbers_list)
    return num

data_3['租期'] = data_2['租期'].apply(process_time)
test_3['租期'] = test_2['租期'].apply(process_time)

In [25]:
# 物业费
def process_cost(s):
    if pd.isna(s):
        return np.nan

    num = np.nan
    numbers = re.findall(r'\d+', s)
    if numbers:
        numbers_list = [int(x) for x in numbers]
        num = sum(numbers_list) / len(numbers_list)
    return num

data_3['物业费'] = data_2['物业费'].apply(process_cost)
test_3['物业费'] = test_2['物业费'].apply(process_cost)

In [26]:
# 停车位
data_3['停车位'] = data_2['停车位'].copy()
test_3['停车位'] = test_2['停车位'].copy()

In [27]:
print(data_3.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97827 entries, 0 to 97826
Data columns (total 54 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   城市      97827 non-null  int64  
 1   区县      94222 non-null  float64
 2   板块      93755 non-null  float64
 3   环线位置    29236 non-null  float64
 4   Price   97827 non-null  float64
 5   室       97793 non-null  float64
 6   厅       92799 non-null  float64
 7   卫       34229 non-null  float64
 8   所在楼层    97822 non-null  float64
 9   总层数     97815 non-null  float64
 10  面积      97827 non-null  float64
 11  朝向_东    97822 non-null  float64
 12  朝向_南    97822 non-null  float64
 13  朝向_西    97822 non-null  float64
 14  朝向_北    97822 non-null  float64
 15  朝向_东南   97822 non-null  float64
 16  朝向_东北   97822 non-null  float64
 17  朝向_西南   97822 non-null  float64
 18  朝向_西北   97822 non-null  float64
 19  装修      97827 non-null  object 
 20  付款方式    97827 non-null  object 
 21  租赁方式    97827 non-null  object 
 22

In [28]:
print(test_3.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9773 entries, 0 to 9772
Data columns (total 53 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   城市      9773 non-null   int64  
 1   区县      8848 non-null   float64
 2   板块      8837 non-null   float64
 3   环线位置    2811 non-null   float64
 4   室       9761 non-null   float64
 5   厅       9644 non-null   float64
 6   卫       9773 non-null   int64  
 7   所在楼层    9773 non-null   int64  
 8   总层数     9773 non-null   int64  
 9   面积      9773 non-null   float64
 10  朝向_东    9773 non-null   int64  
 11  朝向_南    9773 non-null   int64  
 12  朝向_西    9773 non-null   int64  
 13  朝向_北    9773 non-null   int64  
 14  朝向_东南   9773 non-null   int64  
 15  朝向_东北   9773 non-null   int64  
 16  朝向_西南   9773 non-null   int64  
 17  朝向_西北   9773 non-null   int64  
 18  装修      9773 non-null   object 
 19  付款方式    9773 non-null   object 
 20  租赁方式    9773 non-null   object 
 21  电梯      9773 non-null   object 
 22  

In [29]:
data_3.to_feather('Change_12/data_3_rent.feather')
test_3.to_feather('Change_12/test_3_rent.feather')

### 填充缺失值

In [30]:
'''
# 提取数值型特征和非数值型特征
numerical_cols = ['环线位置', '室', '厅', '卫', '所在楼层', '总层数', '面积', 'lon', 'lat', '年份', '建筑年代_1', '建筑年代_2',
                  '房屋总数', '楼栋总数', '绿化率', '容积率', '租期', '物业费', '停车位']
categorical_cols = ['城市', '区县', '板块', '朝向_东', '朝向_南', '朝向_西', '朝向_北', '朝向_东南', '朝向_东北', '朝向_西南', '朝向_西北',
                    '装修', '付款方式', '租赁方式', '电梯', '用水', '用电', '燃气', '采暖', '交易时间', '洗衣机', '空调', '衣柜', '热水器', 
                    '床', '宽带', '电视', '冰箱', '暖气', '天然', '塔楼', '平房', '板楼', '塔板结合']
'''

"\n# 提取数值型特征和非数值型特征\nnumerical_cols = ['环线位置', '室', '厅', '卫', '所在楼层', '总层数', '面积', 'lon', 'lat', '年份', '建筑年代_1', '建筑年代_2',\n                  '房屋总数', '楼栋总数', '绿化率', '容积率', '租期', '物业费', '停车位']\ncategorical_cols = ['城市', '区县', '板块', '朝向_东', '朝向_南', '朝向_西', '朝向_北', '朝向_东南', '朝向_东北', '朝向_西南', '朝向_西北',\n                    '装修', '付款方式', '租赁方式', '电梯', '用水', '用电', '燃气', '采暖', '交易时间', '洗衣机', '空调', '衣柜', '热水器', \n                    '床', '宽带', '电视', '冰箱', '暖气', '天然', '塔楼', '平房', '板楼', '塔板结合']\n"

In [31]:
# 提取数值型特征和非数值型特征
numerical_cols = data_3.select_dtypes(include=[np.number]).columns.tolist()
numerical_cols.remove('Price')
numerical_cols.remove('城市')
numerical_cols.remove('区县')
numerical_cols.remove('板块')
categorical_cols = data_3.select_dtypes(include=['object', 'category']).columns.tolist()
categorical_cols.append('城市')
categorical_cols.append('区县')
categorical_cols.append('板块')

In [32]:
# 数值型特征填充缺失值：InterativeImputer
imputer = IterativeImputer(
    max_iter=30,
    tol=1e-3,
    initial_strategy='median',  # 初始化策略：median, mean, most_frequent
    imputation_order='ascending',  # 填补顺序：ascending, descending, roman, random
    random_state=42,
    verbose=2
)

In [33]:
data_num = pd.DataFrame(imputer.fit_transform(data_3[numerical_cols]), columns=numerical_cols)
test_num = pd.DataFrame(imputer.transform(test_3[numerical_cols]), columns=numerical_cols)
print(data_num.info())

[IterativeImputer] Completing matrix with shape (97827, 42)
[IterativeImputer] Ending imputation round 1/30, elapsed time 9.55
[IterativeImputer] Change: 4039.5905248296895, scaled tolerance: 12.669 
[IterativeImputer] Ending imputation round 2/30, elapsed time 18.25
[IterativeImputer] Change: 2540.654456780194, scaled tolerance: 12.669 
[IterativeImputer] Ending imputation round 3/30, elapsed time 27.14
[IterativeImputer] Change: 2050.873141660681, scaled tolerance: 12.669 
[IterativeImputer] Ending imputation round 4/30, elapsed time 34.65
[IterativeImputer] Change: 1905.2514841672382, scaled tolerance: 12.669 
[IterativeImputer] Ending imputation round 5/30, elapsed time 42.12
[IterativeImputer] Change: 570.1713448053924, scaled tolerance: 12.669 
[IterativeImputer] Ending imputation round 6/30, elapsed time 49.83
[IterativeImputer] Change: 345.96578244500904, scaled tolerance: 12.669 
[IterativeImputer] Ending imputation round 7/30, elapsed time 57.24
[IterativeImputer] Change: 236

In [34]:
# 非数值型变量填充缺失值：基于NearestNeighbors(用Neighbors的众数填充，若众数为空，则用全局众数填充)
scaler_num = StandardScaler()
data_num_standard = scaler_num.fit_transform(data_num)
test_num_standard = scaler_num.transform(test_num)
k = 100
nn = NearestNeighbors(n_neighbors=k+1)
nn.fit(data_num_standard)
distances_data, indices_data = nn.kneighbors(data_num_standard)
distances_test, indices_test = nn.kneighbors(test_num_standard)

In [35]:
data_cat = data_3[categorical_cols].copy()
test_cat = test_3[categorical_cols].copy()

# 预计算每个非数值型特征的全局众数
global_modes = {}
for col in categorical_cols:
    col_mode = data_cat[col].mode()
    if not col_mode.empty:
        global_modes[col] = col_mode[0]
    else:
        global_modes[col] = np.nan

In [36]:
# 遍历每个非数值型列，填充缺失值
for col in categorical_cols:
    missing_indices = data_cat[data_cat[col].isna()].index
    if len(missing_indices) == 0:
        continue
    
    # 对每个缺失值样本进行填充
    for idx in missing_indices:
        # 获取近邻索引
        neighbor_ids = indices_data[idx][1:]  # indices第一列是样本自身
        # 提取近邻在当前列的非缺失值
        neighbor_values = data_cat.iloc[neighbor_ids][col].dropna()

        # 填充
        if not neighbor_values.empty:
            fill_value = neighbor_values.mode()[0]
        else:
            fill_value = global_modes[col]
        data_cat.loc[idx, col] = fill_value

# 遍历每个非数值型列，填充缺失值
for col in categorical_cols:
    missing_indices = test_cat[test_cat[col].isna()].index
    if len(missing_indices) == 0:
        continue
    
    # 对每个缺失值样本进行填充
    for idx in missing_indices:
        # 获取近邻索引
        neighbor_ids = indices_test[idx]
        # 提取近邻在当前列的非缺失值
        neighbor_values = data_cat.iloc[neighbor_ids][col].dropna()

        # 填充
        if not neighbor_values.empty:
            fill_value = neighbor_values.mode()[0]
        else:
            fill_value = global_modes[col]
        test_cat.loc[idx, col] = fill_value

In [37]:
# 合并数值型与非数值型
data_4 = pd.concat([data_num, data_cat], axis=1)
test_4 = pd.concat([test_num, test_cat], axis=1)
data_4.to_feather('Change_12/data_4_rent.feather')
test_4.to_feather('Change_12/test_4_rent.feather')

### 独热编码分类变量

In [38]:
categorical_cols_2 = [col for col in categorical_cols if col != '城市' and col !='板块']

In [39]:
encoder = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')
d_cat_encoded = encoder.fit_transform(data_cat[categorical_cols_2])
t_cat_encoded = encoder.transform(test_cat[categorical_cols_2])
data_cat_encoded = pd.DataFrame(d_cat_encoded, columns=encoder.get_feature_names_out(categorical_cols_2),index=data_cat.index)
test_cat_encoded = pd.DataFrame(t_cat_encoded, columns=encoder.get_feature_names_out(categorical_cols_2),index=test_cat.index)

/root/miniconda3/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [8] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [40]:
data_5 = pd.concat([data_num, data_cat_encoded], axis=1)
test_5 = pd.concat([test_num, test_cat_encoded], axis=1)
data_5.to_feather('Change_12/data_5_rent.feather')
test_5.to_feather('Change_12/test_5_rent.feather')

In [42]:
def process_outliers(df, columns=None):
    df_clean = df.copy()
    
    if columns is None:
        columns = df.select_dtypes(include=[np.number]).columns
    
    for col in columns:
        # 计算均值和标准差
        mean = df_clean[col].mean()
        std = df_clean[col].std()
        
        # 计算3σ原则的上下界
        lower_bound = mean - 3 * std
        upper_bound = mean + 3 * std
        
        # 删除异常值
        df_clean = df_clean[(df_clean[col] >= lower_bound) & (df_clean[col] <= upper_bound)]
    
    return df_clean

# 删除异常值
data_combined = pd.concat([data_5, data_3['Price']], axis=1)
data_6 = process_outliers(data_combined, ['Price', '所在楼层', '总层数', '面积', '租期', '房屋总数', '楼栋总数', '绿化率', '容积率', '物业费', '停车位'])
print(f"原始数据形状: {data_combined.shape}")
print(f"删除异常值后形状: {data_6.shape}")

原始数据形状: (97827, 158)
删除异常值后形状: (84358, 158)


### 引入交叉项

In [43]:
# 楼层比
data_6['楼层比'] = data_6['所在楼层'] / ( data_6['总层数'] + 1 )
test_5['楼层比'] = test_5['所在楼层'] / ( test_5['总层数'] + 1 )

# 楼房比
data_6['楼房比'] = data_6['楼栋总数'] / data_6['房屋总数']
test_5['楼房比'] = test_5['楼栋总数'] / test_5['房屋总数']

# 容积绿化比
data_6['容积绿化比'] = data_6['容积率'] / data_6['绿化率']
test_5['容积绿化比'] = test_5['容积率'] / test_5['绿化率']

In [44]:
data_6.to_feather('Change_12/data_6.feather')
test_5.to_feather('Change_12/test_6.feather')

## 模型训练与预测

In [45]:
# 划分样本内（训练集）与样本外（测试集）
scaler = StandardScaler()
X = scaler.fit_transform(data_6.drop('Price', axis=1))
X_predict = scaler.transform(test_5)
y = data_6['Price'] / data_6['面积']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=111
)

In [46]:
# 模型定义与超参数网格
models = {
    'OLS': LinearRegression(),
    'Lasso': Lasso(random_state=42),
    'Ridge': Ridge(random_state=42),
    'ElasticNet': ElasticNet(random_state=42),
}

param_grids = {
    'Lasso': {'alpha': [0.01, 0.1, 1, 10, 100]},
    'Ridge': {'alpha': [0.01, 0.1, 1, 10, 100]},
    'ElasticNet': {
        'alpha': [0.01, 0.1, 1, 10],
        'l1_ratio': [0.2, 0.5, 0.8]
    },
}

# 调优函数
def tune_model(model_name, X, y):
    model = models[model_name]
    if model_name == 'OLS':
        model.fit(X, y)
        cv_scores = cross_val_score(
            estimator=model,
            X=X,
            y=y,
            cv=6,
            scoring='neg_mean_absolute_error',
            n_jobs=-1
        )
        mae = -cv_scores.mean()
        return model, None, mae
    else:
        grid_search = GridSearchCV(
            estimator=model,
            param_grid=param_grids[model_name],
            cv=6,
            scoring='neg_mean_absolute_error',
            n_jobs=-1
        )
        grid_search.fit(X, y)
        return grid_search.best_estimator_, grid_search.best_params_, -grid_search.best_score_

In [47]:
# 训练并调优所有模型
tuned_models = {}
for name in models.keys():
    print(f"调优{name}模型...")
    model, best_params, mae = tune_model(name, X_train, y_train)
    tuned_models[name] = {'model': model, 'params': best_params, 'mae': mae}
    print(tuned_models[name])

调优OLS模型...
{'model': LinearRegression(), 'params': None, 'mae': np.float64(1828.5151642949932)}
调优Lasso模型...


/root/miniconda3/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.135e+10, tolerance: 2.199e+08
  model = cd_fast.enet_coordinate_descent(
/root/miniconda3/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.352e+10, tolerance: 2.198e+08
  model = cd_fast.enet_coordinate_descent(
/root/miniconda3/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.512e+10,

{'model': Lasso(alpha=1, random_state=42), 'params': {'alpha': 1}, 'mae': np.float64(1827.6941008637793)}
调优Ridge模型...
{'model': Ridge(alpha=0.1, random_state=42), 'params': {'alpha': 0.1}, 'mae': np.float64(1828.5148030740738)}
调优ElasticNet模型...


/root/miniconda3/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.811e+10, tolerance: 2.205e+08
  model = cd_fast.enet_coordinate_descent(
/root/miniconda3/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.658e+10, tolerance: 2.201e+08
  model = cd_fast.enet_coordinate_descent(
/root/miniconda3/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.658e+10,

{'model': ElasticNet(alpha=0.1, l1_ratio=0.8, random_state=42), 'params': {'alpha': 0.1, 'l1_ratio': 0.8}, 'mae': np.float64(1828.012652946674)}


In [48]:
# 计算RMAE（MAE除以目标均值）
def rmae(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    return mae / np.mean(y_true)

# 评估函数
def evaluate_model(model_name, X_train, y_train, X_test, y_test):
    model = tuned_models[model_name]['model']
    
    # 样本内预测
    y_pred_train = model.predict(X_train)
    mae_in = mean_absolute_error(y_train, y_pred_train)
    rmae_in = rmae(y_train, y_pred_train)
    
    # 样本外预测
    y_pred_test = model.predict(X_test)
    mae_out = mean_absolute_error(y_test, y_pred_test)
    rmae_out = rmae(y_test, y_pred_test)
    
    # 6折交叉验证
    mae_cv = tuned_models[name]['mae']
    rmae_cv = mae_cv / np.mean(y_train)  # 用训练集均值计算RMAE
    
    return {
        'MAE_in': mae_in, 'RMAE_in': rmae_in,
        'MAE_out': mae_out, 'RMAE_out': rmae_out,
        'MAE_cv': mae_cv, 'RMAE_cv': rmae_cv
    }

In [49]:
# 评估所有模型
results = {}
for name in tuned_models.keys():
    results[name] = evaluate_model(name, X_train, y_train, X_test, y_test)

In [50]:
# 整理报告表格（以MAE为例，RMAE类似）
report = pd.DataFrame(columns=['指标', '样本内', '样本外', '交叉验证'])

# 填充OLS结果
report.loc[0] = [
    'OLS (MAE)', 
    round(results['OLS']['MAE_in'], 2),
    round(results['OLS']['MAE_out'], 2),
    round(results['OLS']['MAE_cv'], 2)
]

# 填充Lasso结果
report.loc[1] = [
    'Lasso (MAE)', 
    round(results['Lasso']['MAE_in'], 2),
    round(results['Lasso']['MAE_out'], 2),
    round(results['Lasso']['MAE_cv'], 2)
]

# 填充Ridge结果
report.loc[2] = [
    'Ridge (MAE)', 
    round(results['Ridge']['MAE_in'], 2),
    round(results['Ridge']['MAE_out'], 2),
    round(results['Ridge']['MAE_cv'], 2)
]

# 填充ElasticNet结果
report.loc[3] = [
    'ElasticNet (MAE)', 
    round(results['ElasticNet']['MAE_in'], 2),
    round(results['ElasticNet']['MAE_out'], 2),
    round(results['ElasticNet']['MAE_cv'], 2)
]


print("\n性能指标报告（MAE）：")
print(report)

# 打印RMAE结果（可选）
print("\nRMAE结果：")
for name in ['OLS', 'Lasso', 'Ridge', 'ElasticNet']:
    print(f"{name} - 样本内: {round(results[name]['RMAE_in'], 4)}, "
          f"样本外: {round(results[name]['RMAE_out'], 4)}, "
          f"交叉验证: {round(results[name]['RMAE_cv'], 4)}")


性能指标报告（MAE）：
                 指标      样本内      样本外     交叉验证
0         OLS (MAE)  1823.63  1815.48  1828.52
1       Lasso (MAE)  1822.89  1814.73  1827.69
2       Ridge (MAE)  1823.63  1815.48  1828.51
3  ElasticNet (MAE)  1823.29  1814.87  1828.01

RMAE结果：
OLS - 样本内: 0.2283, 样本外: 0.2278, 交叉验证: 0.2289
Lasso - 样本内: 0.2282, 样本外: 0.2277, 交叉验证: 0.2288
Ridge - 样本内: 0.2283, 样本外: 0.2278, 交叉验证: 0.2289
ElasticNet - 样本内: 0.2283, 样本外: 0.2277, 交叉验证: 0.2289


In [51]:
# 查看模型在整个训练数据上的指标
X_all = data_5.copy()
X_all['楼层比'] = X_all['所在楼层'] / ( X_all['总层数'] + 1 )
X_all['楼房比'] = X_all['楼栋总数'] / X_all['房屋总数']
X_all['容积绿化比'] = X_all['容积率'] / X_all['绿化率']
X_all = scaler.transform(X_all)
y_all = data_3['Price']

In [52]:
for name in tuned_models.keys():
    model = tuned_models[name]['model']
    y_pred = model.predict(X_all) * data_5['面积']
    mae1 = mean_absolute_error(y_all, y_pred)
    rmae1 = mae1/np.mean(y_all)
    mse1 = mean_squared_error(y_all, y_pred)
    rmse1 = np.sqrt(mse1)
    r21 = r2_score(y_all, y_pred)
    print('\n', model)
    print('mae:', mae1)
    print('rmae:', rmae1)
    print('mse:', mse1)
    print('rmse:', rmse1)
    print('r2:', r21)


 LinearRegression()
mae: 172329.8316194318
rmae: 0.29375620307434414
mse: 193673521721.56815
rmse: 440083.53948036744
r2: 0.5025308180233564

 Lasso(alpha=1, random_state=42)
mae: 172344.91885873073
rmae: 0.2937819210251463
mse: 193810024858.46295
rmse: 440238.5999187974
r2: 0.5021801965070816

 Ridge(alpha=0.1, random_state=42)
mae: 172330.80998211095
rmae: 0.29375787080710497
mse: 193673865138.86212
rmse: 440083.9296530403
r2: 0.502529935922805

 ElasticNet(alpha=0.1, l1_ratio=0.8, random_state=42)
mae: 172013.20164948935
rmae: 0.2932164705342739
mse: 192976270766.31567
rmse: 439290.6449792844
r2: 0.5043217745736305


In [53]:
for name in tuned_models.keys():
    model = tuned_models[name]['model']
    y_pred = model.predict(X_predict) * test_5['面积']
    
    # 创建结果DataFrame
    result_df = pd.DataFrame({
        'ID': original_test['ID'],
        'Price': y_pred
    })
    
    # 保存为CSV文件
    filename = f'Change_12/{name}_predictions_rent.csv'
    result_df.to_csv(filename, index=False)
    print(f"{name} 预测结果已保存到 {filename}")

OLS 预测结果已保存到 Change_12/OLS_predictions_rent.csv
Lasso 预测结果已保存到 Change_12/Lasso_predictions_rent.csv
Ridge 预测结果已保存到 Change_12/Ridge_predictions_rent.csv
ElasticNet 预测结果已保存到 Change_12/ElasticNet_predictions_rent.csv
